# Data Preperation

In this notebook, we use a subset of Stack Exchange data (provide citation here) which includes original questions tagged as "JavaScript", their duplicate questions and the answers. Here we provide the steps to prepare the data to later use in model development to train a model that will match a new question with an exisiting original question. 

In [1]:
import os
import pandas as pd
import re
import gzip
import requests
import math
import timeit

In [2]:
test_size = 0.21
min_text = 150
min_dupes = 12 
match = 20

In [3]:
def read_csv_gz(url, **kwargs):
    """Load raw data from a .tsv.gz file into Pandas data frame."""
    df = pd.read_csv(gzip.open(requests.get(url, stream=True).raw, mode='rb'),
                     sep='\t', encoding='utf8', **kwargs)
    return df.set_index('Id')

In [4]:
def clean_text(text):
    """Remove embedded code chunks, HTML tags and links/URLs."""
    if not isinstance(text, str):
        return text
    text = re.sub('<pre><code>.*?</code></pre>', '', text)
    text = re.sub('<a[^>]+>(.*)</a>', replace_link, text)
    return re.sub('<[^>]+>', '', text)

In [5]:
def replace_link(match):
    if re.match('[a-z]+://', match.group(1)):
        return ''
    else:
        return match.group(1)

In [6]:
def round_sample(X, frac=0.1, min=1):
    """Sample X ensuring at least min samples are selected."""
    n = max(min, math.floor(len(X) * frac))
    return X.sample(n)

In [7]:
def round_sample_strat(X, strat, **kwargs):
    """Sample X ensuring at least min samples are selected."""
    return X.groupby(strat).apply(round_sample, **kwargs)

In [8]:
def random_merge(A, B, N=20, on='AnswerId', key='key', n='n'):
    assert key not in A and key not in B
    X = A.copy()
    X[key] = A[on]
    Y = B.copy()
    Y[key] = B[on]
    match = X.merge(Y, on=key).drop(key, axis=1)
    match[n] = 0
    df_list = [match]
    for i in A.index:
        X = A.loc[[i]]
        Y = B[B[on] != X[on].iloc[0]].sample(N-1)
        X[key] = 1
        Y[key] = 1
        Z = X.merge(Y, how='outer', on=key).drop(key, axis=1)
        Z[n] = range(1, N)
        df_list.append(Z)
    df = pd.concat(df_list, ignore_index=True)
    return df

In [9]:
# URLs to original questions, duplicate questions, and answers.
data_url = 'https://bostondata.blob.core.windows.net/stackoverflow/{}'
questions_url = data_url.format('orig-q.tsv.gz')
dupes_url = data_url.format('dup-q.tsv.gz')
answers_url = data_url.format('ans.tsv.gz')

In [10]:
# Load datasets.
questions = read_csv_gz(questions_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
dupes = read_csv_gz(dupes_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
answers = read_csv_gz(answers_url, names=('Id', 'Text0'))

In [11]:
questions.head()

,AnswerId,Text0,CreationDate
Id,,,
220231,220233,Accessing the web page's HTTP Headers in JavaS...,2008-10-20 22:54:38.767
391979,810461,Get client IP using just JavaScript?. <p>I nee...,2008-12-24 18:22:30.780
109086,109091,Stop setInterval call in JavaScript. <p>I am u...,2008-09-20 19:29:55.377
46155,46181,Validate email address in JavaScript?. <p>How ...,2008-09-05 16:10:11.093
121499,121708,"When onblur occurs, how can I find out which e...",2008-09-23 14:48:43.483


In [12]:
dupes.head()

,AnswerId,Text0,CreationDate
Id,,,
665430,665404,"Disable ""Back"" & ""Refresh"" Button in Browser. ...",2009-03-20 09:13:31.800
114525,336868,"The difference between the two functions? (""fu...",2008-09-22 12:24:06.583
1347093,147765,ASP.NET Page_Unload to stop user from leaving ...,2009-08-28 13:46:51.217
1208252,26633883,See if a variable is an array using JavaScript...,2009-07-30 17:57:42.363
177867,122704,How do I copy the data of an element with jque...,2008-10-07 10:23:40.017


In [13]:
answers.head()

,Text0
Id,
119473,"<p>Try <a href=""http://johannburkard.de/blog/p..."
324533,"<p>Adapted from <a href=""http://www.javascript..."
108232,"<p>That is known as a textbox watermark, and i..."
194399,<p><strong>Obfuscation:</strong></p> <p>Try <a...
80127,"<p>In JavaScript, ""this"" always refers to the ..."


In [14]:
questions.iloc[0,1]

'Accessing the web page\'s HTTP Headers in JavaScript. <p>How do I access a page\'s HTTP response headers via JavaScript?</p> <p>Related to <a href="http://stackoverflow.com/questions/220149/how-do-i-access-the-http-request-header-fields-via-javascript"><strong>this question</strong></a>, which was modified to ask about accessing two specific HTTP headers.</p> <blockquote> <p><strong>Related:</strong><br> <a href="http://stackoverflow.com/questions/220149/how-do-i-access-the-http-request-header-fields-via-javascript">How do I access the HTTP request header fields via JavaScript?</a></p> </blockquote>'

In [15]:
dupes[dupes.AnswerId == questions.iloc[0,0]]

,AnswerId,Text0,CreationDate
Id,,,
3177208,220233,Monitoring http request header on a page. <blo...,2010-07-05 04:20:19.663
12258705,220233,How can I read the current headers without mak...,2012-09-04 07:31:07.973
12256134,220233,How to know mime-type or content-type of curre...,2012-09-04 02:43:08.860
15135883,220233,How to access http response headers. <pre><cod...,2013-02-28 12:44:38.393
14673437,220233,Translate Prototype into jQuery?. <blockquote>...,2013-02-03 14:19:00.697
17466305,220233,How to read HTTP header values from JavaScript...,2013-07-04 09:08:32.240
26647511,220233,Is there a JS API to get information about hea...,2014-10-30 07:43:01.117
35604233,220233,How to read http request headers with javascri...,2016-02-24 14:00:49.247


In [16]:
answers.at[questions.iloc[0,0],'Text0']

'<p>Unfortunately, there isn\'t an API to give you the HTTP response headers for your initial page request. That was the original question posted here. It has been <a href="http://stackoverflow.com/questions/12258705/how-can-i-read-the-current-headers-without-making-a-new-request-with-js">repeatedly asked</a>, too, because some people would like to get the actual response headers of the original page request without issuing another one.</p> <h1><br/>For AJAX Requests:</h1> <p>If an HTTP request is made over AJAX, it is possible to get the response headers with the <strong><code>getAllResponseHeaders()</code></strong> method. It\'s part of the XMLHttpRequest API. To see how this can be applied, check out the <em><code>fetchSimilarHeaders()</code></em> function below. Note that this is a work-around to the problem that won\'t be reliable for some applications.</p> <pre><code>myXMLHttpRequest.getAllResponseHeaders(); </code></pre> <ul> <li><p>The API was specified in the following candida

In [17]:
# Clean up all text, and keep only data with some clean text.
for df in (questions, dupes, answers):
    df['Text'] = df.Text0.apply(clean_text).str.lower()

In [18]:
questions = questions[questions.Text.str.len() > 0]
answers = answers[answers.Text.str.len() > 0]
dupes = dupes[dupes.Text.str.len() > 0]

In [19]:
questions.iloc[0,1]

'Accessing the web page\'s HTTP Headers in JavaScript. <p>How do I access a page\'s HTTP response headers via JavaScript?</p> <p>Related to <a href="http://stackoverflow.com/questions/220149/how-do-i-access-the-http-request-header-fields-via-javascript"><strong>this question</strong></a>, which was modified to ask about accessing two specific HTTP headers.</p> <blockquote> <p><strong>Related:</strong><br> <a href="http://stackoverflow.com/questions/220149/how-do-i-access-the-http-request-header-fields-via-javascript">How do I access the HTTP request header fields via JavaScript?</a></p> </blockquote>'

In [20]:
questions.iloc[0,3]

"accessing the web page's http headers in javascript. how do i access a page's http response headers via javascript? related to this question, which was modified to ask about accessing two specific http headers.  related: how do i access the http request header fields via javascript? "

In [21]:
# First, remove dupes that are questions, then remove duplicated
# questions and dupes.
dupes = dupes[~dupes.index.isin(questions.index)]
questions = questions[~questions.index.duplicated(keep='first')]
dupes = dupes[~dupes.index.duplicated(keep='first')]

In [22]:
# Keep only questions with answers and dupes, answers to
# questions, and dupes of questions.
questions = questions[questions.AnswerId.isin(answers.index) & questions.AnswerId.isin(dupes.AnswerId)]
answers = answers[answers.index.isin(questions.AnswerId)]
dupes = dupes[dupes.AnswerId.isin(questions.AnswerId)]

In [23]:
# Verify data integrity.
assert questions.AnswerId.isin(answers.index).all()
assert answers.index.isin(questions.AnswerId).all()
assert questions.AnswerId.isin(dupes.AnswerId).all()
assert dupes.AnswerId.isin(questions.AnswerId).all()

In [24]:
# Report on the data.
print('Text statistics:')
print(pd.DataFrame([questions.Text.str.len().describe()
                    .rename('questions'),
                    answers.Text.str.len().describe()
                    .rename('answers'),
                    dupes.Text.str.len().describe()
                    .rename('dupes')]))
print('\nDuplication statistics:')
print(pd.DataFrame([dupes.AnswerId.value_counts().describe()
                    .rename('duplications')]))
print('\nLargest class: {:.2%}'.format(
    dupes.AnswerId.value_counts().max()
    / dupes.shape[0]))


Text statistics:
             count        mean         std   min    25%    50%    75%     max
questions   1714.0  415.827305  319.857854  56.0  225.0  334.0  509.0  3982.0
answers     1714.0  616.274212  673.060199   1.0  178.0  375.0  757.0  3982.0
dupes      16139.0  441.303612  363.638297  25.0  247.0  357.0  519.0  3989.0

Duplication statistics:
               count      mean        std  min  25%  50%  75%     max
duplications  1714.0  9.415986  41.638847  1.0  3.0  4.0  7.0  1369.0

Largest class: 8.48%


In [25]:
# Reset each dataframe's index.
questions.reset_index(inplace=True)
answers.reset_index(inplace=True)
dupes.reset_index(inplace=True)

In [26]:
# Apply the minimum text length to questions and dupes.
questions = questions[questions.Text.str.len() >= min_text]
dupes = dupes[dupes.Text.str.len() >= min_text]

In [27]:
# Keep only questions with dupes, and dupes of questions.
label_column = 'AnswerId'
questions = questions[questions[label_column].isin(dupes[label_column])]
dupes = dupes[dupes[label_column].isin(questions[label_column])]

In [28]:
# Restrict the questions to those with a minimum number of dupes.
answerid_count = dupes.groupby(label_column)[label_column].count()
answerid_min = answerid_count.index[answerid_count >= min_dupes]
questions = questions[questions[label_column].isin(answerid_min)]
dupes = dupes[dupes[label_column].isin(answerid_min)]

In [29]:
 # Verify data integrity.
assert questions[label_column].isin(dupes[label_column]).all()
assert dupes[label_column].isin(questions[label_column]).all()

In [30]:
# Report on the data.
print('Restrictions: min_text={}, min_dupes={}'.format(
    min_text, min_dupes))
print('Restricted text statistics:')
print(pd.DataFrame([questions.Text.str.len().describe()
                    .rename('questions'),
                    dupes.Text.str.len().describe()
                    .rename('dupes')]))
print('\nRestricted duplication statistics:')
print(pd.DataFrame([dupes[label_column].value_counts().describe()
                    .rename('duplications')]))
print('\nRestricted largest class: {:.2%}'.format(
    dupes[label_column].value_counts().max()
    / dupes.shape[0]))

Restrictions: min_text=150, min_dupes=12
Restricted text statistics:
            count        mean         std    min     25%    50%    75%     max
questions   182.0  413.450549  218.028193  153.0  264.25  338.5  510.5  1475.0
dupes      8260.0  479.882324  398.791447  150.0  270.00  380.0  553.0  3989.0

Restricted duplication statistics:
              count       mean         std   min   25%   50%   75%     max
duplications  182.0  45.384615  117.074823  12.0  15.0  20.0  33.0  1328.0

Restricted largest class: 16.08%


In [31]:
# Split dupes into train and test ensuring at least one of each
# label class is in test.
dupes_test = round_sample_strat(dupes, dupes[label_column], frac=test_size)
dupes_train = dupes[~dupes.Id.isin(dupes_test.Id)]

In [32]:
assert (dupes_test[label_column].unique().shape[0] == dupes[label_column].unique().shape[0])

In [33]:
# The relevant columns for text pairs data.
balanced_pairs_columns = ['Id_x', 'AnswerId_x', 'Text_x', 'Id_y', 'Text_y', 'AnswerId_y', 'Label', 'n']

In [34]:
%%time
# Use AnswerId to pair each training dupe with its matching question and also with N-1 questions not its match.
balanced_pairs_train = random_merge(dupes_train, questions, N=match)

CPU times: user 42.8 s, sys: 125 ms, total: 43 s
Wall time: 43 s


In [35]:
# Label records by matching AnswerIds.
balanced_pairs_train['Label'] = (balanced_pairs_train.AnswerId_x == balanced_pairs_train.AnswerId_y).astype(int)

In [36]:
# Keep only the relevant data.
balanced_pairs_train = balanced_pairs_train[balanced_pairs_columns]

In [37]:
balanced_pairs_train

,Id_x,AnswerId_x,Text_x,Id_y,Text_y,AnswerId_y,Label,n
0,177867,122704,how do i copy the data of an element with jque...,122102,what is the most efficient way to clone an obj...,122704,1,0
1,565430,122704,(deep) copying an array using jquery. possibl...,122102,what is the most efficient way to clone an obj...,122704,1,0
2,3474697,122704,how to clone js object?. possible duplicate: ...,122102,what is the most efficient way to clone an obj...,122704,1,0
3,9610918,122704,how do i get a new reference to an object. po...,122102,what is the most efficient way to clone an obj...,122704,1,0
4,12690107,122704,clone object without reference javascript. i h...,122102,what is the most efficient way to clone an obj...,122704,1,0
5,13756482,122704,"create copy of multi-dimensional array, not re...",122102,what is the most efficient way to clone an obj...,122704,1,0
6,17544977,122704,"creating a new object, not a reference. this i...",122102,what is the most efficient way to clone an obj...,122704,1,0
7,18076125,122704,javascript variables keep track to their copie...,122102,what is the most efficient way to clone an obj...,122704,1,0
8,17416770,122704,"set a variable equal to a value, not a referen...",122102,what is the most efficient way to clone an obj...,122704,1,0
9,17228869,122704,copy a new js object. i have object listpoint...,122102,what is the most efficient way to clone an obj...,122704,1,0


In [38]:
# Sort the data by dupe ID and Label.
balanced_pairs_train.sort_values(by=['Id_x', 'Label'], ascending=[True, False], inplace=True)

In [39]:
%%time
# Use AnswerId to pair each testing dupe with all questions.
balanced_pairs_test = random_merge(dupes_test, questions, N=questions.shape[0])

CPU times: user 12.4 s, sys: 12.1 ms, total: 12.4 s
Wall time: 12.4 s


In [40]:
# Label records by matching AnswerIds.
balanced_pairs_test['Label'] = (balanced_pairs_test.AnswerId_x == balanced_pairs_test.AnswerId_y).astype(int)

In [41]:
# Keep only the relevant data.
balanced_pairs_test = balanced_pairs_test[balanced_pairs_columns]

In [42]:
balanced_pairs_test

,Id_x,AnswerId_x,Text_x,Id_y,Text_y,AnswerId_y,Label,n
0,8702219,6700,how to get javascript hash table count?. poss...,5223,"length of a javascript object (that is, associ...",6700,1,0
1,7193530,6700,check length of multidimensional arrays with j...,5223,"length of a javascript object (that is, associ...",6700,1,0
2,30533351,27943,filter list of cities with latitude and longit...,27928,calculate distance between two latitude-longit...,27943,1,0
3,34864986,27943,distance between 2 decimal points. i have a pr...,27928,calculate distance between two latitude-longit...,27943,1,0
4,23563684,31047,how to check that a div exists jquery and if i...,31044,"is there an ""exists"" function for jquery?. how...",31047,1,0
5,6899175,31047,check if a div exists with jquery. possible d...,31044,"is there an ""exists"" function for jquery?. how...",31047,1,0
6,18693899,69984,is it needed to close the tag when it's used ...,69913,why don't self-closing script tags work?. what...,69984,1,0
7,6600753,69984,does anyone know why doesn't work. possible ...,69913,why don't self-closing script tags work?. what...,69984,1,0
8,34729059,69984,why isn't self closing?. i've been wondering ...,69913,why don't self-closing script tags work?. what...,69984,1,0
9,9528913,69984,closing tag in html. possible duplicate: why ...,69913,why don't self-closing script tags work?. what...,69984,1,0


In [43]:
# Sort the data by dupe ID and Label.
balanced_pairs_test.sort_values(by=['Id_x', 'Label'], ascending=[True, False], inplace=True)

In [44]:
# Report on the datasets.
print('balanced_pairs_train: {:,} rows with {:.2%} matches'.format(balanced_pairs_train.shape[0], 
                                                                   balanced_pairs_train.Label.mean()))
print('balanced_pairs_test: {:,} rows with {:.2%} matches'.format(balanced_pairs_test.shape[0], 
                                                                  balanced_pairs_test.Label.mean()))

balanced_pairs_train: 132,500 rows with 5.00% matches
balanced_pairs_test: 297,570 rows with 0.55% matches


In [45]:
# Save the data.
balanced_pairs_train_path = 'balanced_pairs_train.tsv'
print('Writing {:,} to {}'.format(balanced_pairs_train.shape[0], balanced_pairs_train_path))
balanced_pairs_train.to_csv(balanced_pairs_train_path, sep='\t',header=True, index=False)

balanced_pairs_test_path = 'balanced_pairs_test.tsv'
print('Writing {:,} to {}'.format(balanced_pairs_test.shape[0], balanced_pairs_test_path))
balanced_pairs_test.to_csv(balanced_pairs_test_path, sep='\t', header=True, index=False)

Writing 132,500 to balanced_pairs_train.tsv
Writing 297,570 to balanced_pairs_test.tsv
